## Notes:
----------

Theory:
We have see a lot of discreapancy with the images, hence for testing puspose we only select images that were last_reviewed after September 2017. We believe that the data atleast reviewed after september should be correct.

1. The accessor photo column had some "nan" values, these were removed.
2. 




## WHAT THE BELOW CELLS DO:

1. We load the metadata file that contains the property metadata for appx 250000 properties including the assessor image.
3. We dont want to experiment on all the images, so we select the ones that were last_reviewed after April 2017. We further filter the images. For land the images that have a improvement level as land are only selected. For House, images with improvement level of anything other than land is taken.
4. We then collect all the assessor images from the links provided in the metadata table. Somtimes the images are not rendered properly, so we filter such images
5. Similarly we gather all the Aerial images using Bing map API call
6. Lastely we acquire all streeside images from Google.
7. We can even gather all the Aerial images using Google map API call

In [2]:
import os
import random
import pandas as pd
import numpy as np
import logging
from semantic_segmentation import utils as utl
from semantic_segmentation.overlay_building_bbox import overlay_parcel_on_images

logging.basicConfig(level=logging.DEBUG, filename="logfile.log", filemode="w", 
                    format="%(asctime)-15s %(levelname)-8s %(message)s")

from external_data import google_aerial # bing_aerial, google_streetside, 
from config import pathDict

parent_path = pathDict['parent_path']
xlsx_file = os.path.join(parent_path, "Image_Classification_Project.xlsx")
meta_data_path = os.path.join(parent_path, "house_metadata_nw.csv")
# assessor_image_path = os.path.join(parent_path, "input_images", "assessor_images")
# # bing_aerial_image_path = os.path.join(parent_path, 'input_images','bing','aerial_images')
# google_aerial_image_path = os.path.join(parent_path, 'input_images','google','aerial_images')
# streetside_image_path = os.path.join(parent_path, 'input_images','streetside_images')

print('Metadata path: ', meta_data_path)
# print('Assessor image path: ', assessor_image_path)
# # print('Aerial image path (Bing): ', bing_aerial_image_path)
# print('Streetside image path: ', streetside_image_path)
# print('Aerial image path (Google): ', google_aerial_image_path)

xlsx_to_csv = False
dump_aerial_google_images  = True
overlay_bbox = True

INPUT the RUN NAME: Options : ("A new run name" or "Any Previous Run Name"
 WHICH_RUN = sam_new
INPUT: Image-type OPTIONS: (assessor, assessor_code, aerial, overlayed, aerial_cropped, streetside and ensemble 
 IMAGE_TYPE = aerial
Metadata path:  C:\Users\newline\Documents\ImageClassification\data\house_metadata_nw.csv


In [13]:
pathDict

{'batch_path': 'C:\\Users\\newline\\Documents\\ImageClassification\\data\\batch_data\\sam_new\\aerial',
 'checkpoint_path': 'C:\\Users\\newline\\Documents\\ImageClassification\\data\\checkpoints\\sam_new\\aerial',
 'chicago_bbox_shp_files': 'C:\\Users\\newline\\Documents\\ImageClassification\\data\\shape_files\\building_bbox',
 'general_stats_path': 'C:\\Users\\newline\\Documents\\ImageClassification\\data\\statistics\\sam_new',
 'image_path': 'C:\\Users\\newline\\Documents\\ImageClassification\\data\\input_images\\sam_new\\aerial',
 'input_image_run_dir': 'C:\\Users\\newline\\Documents\\ImageClassification\\data\\input_images\\sam_new',
 'parent_path': 'C:\\Users\\newline\\Documents\\ImageClassification\\data',
 'statistics_path': 'C:\\Users\\newline\\Documents\\ImageClassification\\data\\statistics\\sam_new\\aerial',
 'summary_path': 'C:\\Users\\newline\\Documents\\ImageClassification\\data\\summary\\sam_new\\aerial'}

In [2]:
pathDict

{'batch_path': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data/batch_data/sam_new/aerial',
 'checkpoint_path': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data/checkpoints/sam_new/aerial',
 'chicago_bbox_shp_files': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data/shape_files/building_bbox',
 'general_stats_path': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data/statistics/sam_new',
 'image_path': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data/input_images/sam_new/aerial',
 'input_image_run_dir': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data/input_images/sam_new',
 'parent_path': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data',
 'statistics_path': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data/statistics/sam_new/aerial',
 'summary_path': '/Users/sam/All-Program/App-DataSet/HouseClassification/training_data/summ

### LOAD THE XLSX FILE GET DATA AND PUT INTO A NEW CSV FILE

In [3]:
if xlsx_to_csv:
    data_xls = pd.read_excel(xlsx_file, 'Data Set', index_col=None)
    data_xls.to_csv(meta_data_path, encoding='utf-8')

# STEP 1: Load Metadata file
------------

In [3]:
metadata = pd.read_csv(meta_data_path)
metadata.columns = ['row_id', 'removed', 'property_id', 'state', 'county_name', 'pin',
       'address_line1', 'address_line2', 'address_city', 'address_zip',
       'zoning', 'improvement_level', 'type', 'exterior',
       'last_reviewed_timestamp', 'gone_timestamp', 'indicator',
       'assessor_photo']
metadata['state'] = metadata['state'].astype('str')
metadata['county_name'] = metadata['county_name'].astype('str')
metadata['pin'] = metadata['pin'].astype('str')
metadata['address_line1'] = metadata['address_line1'].astype('str')
metadata['address_line2'] = metadata['address_line2'].astype('str')
metadata['address_city'] = metadata['address_city'].astype('str')
metadata['address_zip'] = metadata['address_zip'].astype('str')
metadata['zoning'] = metadata['zoning'].astype('str')
metadata['improvement_level'] = metadata['improvement_level'].astype('str')
metadata['type'] = metadata['type'].astype('str')
metadata['exterior'] = metadata['exterior'].astype('str')
metadata['last_reviewed_timestamp'] = pd.to_datetime(metadata['last_reviewed_timestamp'])#.astype('str')
metadata['gone_timestamp'] = pd.to_datetime(metadata['gone_timestamp'])#.astype('str')
metadata['indicator'] = metadata['indicator'].astype('str')
metadata['assessor_photo'] = metadata['assessor_photo'].astype('str')

# Remove the Test Dat from Metadata
metadata = metadata[metadata['removed'] == 0]
print(metadata.shape)
metadata.head(3)

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(220364, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
1,1,0,499849,IL,Cook,01-01-100-109-0000,200S HOUGH ST,nan,BARRINGTON,nan,Vacant,Land,nan,nan,NaT,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
2,2,0,499852,IL,Cook,01-01-101-029-0000,201 First,22 # Floor,BARRINGTON,nan,Vacant,Land,4 Unit Apt Bldg,Frame,NaT,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
3,3,0,499854,IL,Cook,01-01-109-001-0000,323 MAIN ST,nan,BARRINGTON,600104305.0,Residential,Improved,2 Flat,Frame,2017-05-25 04:42:26.850,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


# STEP 2: Create a new Subset
--------------

In [4]:
metadata_lst_rvd_valid = metadata[metadata['last_reviewed_timestamp'] != '']
metadata_lst_rvd_valid['last_reviewed_timestamp'] = pd.to_datetime(metadata_lst_rvd_valid['last_reviewed_timestamp'])
metadata_lst_rvd_valid_2017 = metadata_lst_rvd_valid[((metadata_lst_rvd_valid['last_reviewed_timestamp'].dt.year == 2017) & (metadata_lst_rvd_valid['last_reviewed_timestamp'].dt.month >=1 ) &
                                                      (metadata_lst_rvd_valid['address_city'].str.lower().str.strip().str.match('chicago')) &
                                                      (metadata_lst_rvd_valid['address_line1'] != 'nan') &
                                                      (metadata_lst_rvd_valid['assessor_photo'] != 'nan') &
                                                     (metadata_lst_rvd_valid['indicator'].isin(["Likely House", "Likely Land"]) ))]
image_data = metadata_lst_rvd_valid_2017
print (metadata_lst_rvd_valid_2017.shape)
metadata_lst_rvd_valid_2017.head(2)

(28527, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
6111,6111,0,506508,IL,Cook,09-25-417-038-0000,7334 N OLEANDER AVE,nan,CHICAGO,606314308.0,Residential,Improved,nan,nan,2017-02-15 21:27:08.760,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8746,8746,0,509513,IL,Cook,10-25-300-045-0000,3125 W HOWARD ST,nan,CHICAGO,606451139.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


### LAND DATA
----------------

In [5]:
land_data = image_data[(image_data['indicator'] == 'Likely Land') & (image_data['improvement_level'] == 'Land')]
print (np.unique(np.array(land_data['improvement_level'])))
print (land_data.shape)
land_data.head(3)

['Land']
(9873, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
8746,8746,0,509513,IL,Cook,10-25-300-045-0000,3125 W HOWARD ST,nan,CHICAGO,606451139.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
9339,9339,0,510240,IL,Cook,10-31-417-045-0000,6546 W DEVON AVE,nan,CHICAGO,606312038.0,Vacant,Land,nan,nan,2017-06-22 18:07:04.937,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
9366,9366,0,510267,IL,Cook,10-32-126-007-0000,7046N MCALPIN AVE,nan,CHICAGO,60646.0,Vacant,Land,nan,nan,2017-06-09 22:51:31.103,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


### HOUSE DATA
--------------

In [6]:
house_data = image_data[(image_data['indicator'] == 'Likely House') & (image_data['improvement_level'] != 'Land')]
print (np.unique(np.array(house_data['improvement_level'])))
print (house_data.shape)
house_data.head(3)

['Condominium' 'Improved' 'Minor Improvement' 'Mixed Use']
(18392, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
6111,6111,0,506508,IL,Cook,09-25-417-038-0000,7334 N OLEANDER AVE,nan,CHICAGO,606314308.0,Residential,Improved,nan,nan,2017-02-15 21:27:08.760,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8761,8761,0,509531,IL,Cook,10-25-308-043-0000,3146W FARGO AVE,nan,CHICAGO,60645.0,Residential,Improved,nan,nan,2017-02-10 21:21:30.520,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8769,8769,0,509539,IL,Cook,10-25-314-003-0000,2949W FARGO AVE,nan,CHICAGO,60645.0,Residential,Improved,nan,nan,2017-02-23 16:25:51.033,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


### UNKNOWN DATA
-------------

In [7]:
unknown_data = image_data[image_data['indicator'] == 'Unknown']
print (unknown_data.shape)
unknown_data.head(3)

(0, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo


# Step 1: Load Aerial Images (Google Static Maps):
------------

### Remove from the land_house_data where the PIN is already processed

In [ ]:
from data_transformation.data_load import GetAerial
if dump_aerial_google_images:
    land_house_data = pd.concat([land_data, house_data])
    print (land_house_data.shape)
    pd.concat([land_house_data.head(2), land_house_data.tail(2)])
    GetAerial(zoom=20, state='IL', map_size = '400x400', batch_size=100).parse_dump_aerial(land_house_data)

(28265, 18)
Number of images already parsed and dumped  23556
Number of new images to be parsed and dumped  4709


# Step 2: Create Overlayed Images from Aerial Images:
------------

In [8]:
from data_transformation.data_load import GetOverlayed
obj_overlayed = GetOverlayed()
print (obj_overlayed.overlayed_image_path)
print (obj_overlayed.aerial_image_path)
print (obj_overlayed.stats_path)
obj_overlayed.parse_dump_overlayed()

C:\Users\newline\Documents\ImageClassification\data\input_images\sam_new\overlayed
C:\Users\newline\Documents\ImageClassification\data\input_images\sam_new\aerial
C:\Users\newline\Documents\ImageClassification\data\statistics\sam_new\aerial\aerial_collected_data_stats
Initiating Run for scoop scoop_738_768 : DATA SHAPE (93246, 13)
Total images to be processed are  (0, 9)

Initiating Run for scoop scoop_768_798 : DATA SHAPE (94331, 13)
Total images to be processed are  (1, 9)

Initiating Run for scoop scoop_798_848 : DATA SHAPE (80697, 13)
Total images to be processed are  (0, 9)

Initiating Run for scoop scoop_848_908 : DATA SHAPE (90177, 13)
Total images to be processed are  (0, 9)

Initiating Run for scoop scoop_908_938 : DATA SHAPE (92080, 13)
Total images to be processed are  (359, 9)
TOTAL RECORDS PARSED: IMAGES DONE ======== 358
Initiating Run for scoop scoop_938_968 : DATA SHAPE (91240, 13)
Total images to be processed are  (635, 9)
TOTAL RECORDS PARSED: IMAGES DONE ======== 634

# Step 3: Create Cropped Images from Aerial Images:
------------

In [9]:
from data_transformation.data_load import GetAerialCropped
obj_cropped = GetAerialCropped()
print (obj_cropped.aerial_cropped_img_path)
print (obj_cropped.aerial_image_path)
print (obj_cropped.stats_path)
obj_cropped.parse_dump_aerial_cropped()

C:\Users\newline\Documents\ImageClassification\data\input_images\sam_new\aerial_cropped
C:\Users\newline\Documents\ImageClassification\data\input_images\sam_new\aerial
C:\Users\newline\Documents\ImageClassification\data\statistics\sam_new\aerial\aerial_collected_data_stats
Initiating Run for scoop scoop_738_768 : DATA SHAPE (93246, 13)
Initiating Run for scoop scoop_768_798 : DATA SHAPE (94331, 13)
Initiating Run for scoop scoop_798_848 : DATA SHAPE (80697, 13)
Initiating Run for scoop scoop_848_908 : DATA SHAPE (90177, 13)
Initiating Run for scoop scoop_908_938 : DATA SHAPE (92080, 13)
Initiating Run for scoop scoop_938_968 : DATA SHAPE (91240, 13)
Initiating Run for scoop scoop_968_028 : DATA SHAPE (86448, 13)
